# **Setup**
 
Reset the Python environment to clear it of any previously loaded variables, functions, or libraries. Then, import the libraries needed to complete the code Professor Melnikov presented in the video.

In [ ]:
%reset -f
from IPython.core.interactiveshell import InteractiveShell as IS
IS.ast_node_interactivity = "all"    # allows multiple outputs from a cell
import pandas as pd, numpy as np, seaborn as sns, matplotlib.pyplot as plt, nltk

np.set_printoptions(linewidth=10000, precision=4, edgeitems=20, suppress=True)  # numpy print format
pd.set_option('max_rows',100,'max_columns',100,'max_colwidth',100,'precision',2,'display.max_rows',8) # pandas print format

<hr style="border-top: 2px solid #606366; background: transparent;">

# **Review**

## Levenshtein or Edit Distance

Levenshtein, or edit, distance counts the number of edits needed to convert one sequence to another (such as adding, deleting, or replacing). The can be done on a word, a sentence, binary code (sequence of zeros and ones), a DNA sequence of nucleotides A,C,G,T, etc.
    

## Dynamic Programing (DP)

The dynamic programming (DP) algorithm builds a list of lists (or a matrix) with a number of columns and rows matching each element in two paired sequences. To start the comparison between the two sequenses, one column and one row are pre-pended to begin with a matching character (for example, "" or empty character). A typical DP algorithm will repeat the same operation on each subsequence, thereby recursively computing the desired result, which, in this case, is the count of edit operations. 
 
As iterations over elements of a column (row by row) are processed, the required edit operation is determined. The resulting matrix elements are counters of the operations needed thus far. If no new operations are needed for the current two characters (one from each sequence), the counter remains unchanged. If any operation is needed, the counter is incremented. 

While the algorithim can be hard to understand, it is a fairly simple algorithim in which all of the decisions are done inside conditional statements. The function `EditDist` applies the algorithm and returns the resulting matrix of edit counts.

In [ ]:
def EditDist(s1='', s2='', ShowMatrix=False):
    '''Dynamic programming implementation builds a matrix M (or list of lists) 
    of best edits from sub-problems on substrings'''
    if len(s1) > len(s2): s1, s2 = s2, s1  # keep the shortest word in s1 for matrix display convenience
    n1, n2 = len(s1), len(s2)
    M = [[0 for x in range(n2 + 1)] for x in range(n1 + 1)] # stores results of subproblems

    for i in range(n1 + 1):    # row iterator for the shorter word
        for j in range(n2 + 1):  # column iterator for the longer word
            if   i == 0: M[i][j] = j  # s1 is empty => j ops to copy remaining s2 substring to s1
            elif j == 0: M[i][j] = i  # s2 is empty => i ops to copy remaining s1 substring to s2
            elif s1[i-1] == s2[j-1]: M[i][j] = M[i-1][j-1]  # If prev chars match, ignore these and proceed
            else:  # If prev chars differ, consider all 3 ops on s1 and keep the best operation
                nIns, nRem, nSub = M[i][j-1], M[i-1][j], M[i-1][j-1]  # insertion, removal, substitution
                M[i][j] = 1 + min(nIns, nRem, nSub)
    if ShowMatrix:
        df = pd.DataFrame(M, index=['']+list(s1), columns=['']+list(s2))
        ax = sns.heatmap(df, annot=True, cbar=False, cmap='coolwarm');
        ax.set_title('Best edit distances on substrings')
        ax.xaxis.tick_top()    # x axis on top
        ax.xaxis.set_label_position('top')
        plt.yticks(rotation=0)
    return M[n1][n2]

## Example Matrix

Read the matrix below from left to right, top to bottom. 

1. Row 0:
    1. Cell (0,0) has 0 edits since the corresponding row & column elements already match (both are `''`).
    1. Cell (0,1) has 1 edit since it takes one operation to remove column index `'r'`. This updates cell (0,1) to `''`, which then matches cell (0,0).
    1. For each remaining cell, DP confirms no match between characters and increases the counter.

1. Row 1:
    1. DP starts with a mismatch between `'c'` and `''` which restarts the counter at 1 edit operation.
    1. DP continues, but increments only the smallest of the accessible previous edit counts, selected from the left of the current cell, above the current cell, or diagonally of the current cell.
1. The bottom right corner will contain the final (smallest) count of the required edit operations needed to convert one string into another.

In [ ]:
EditDist('cat', 'rats', ShowMatrix=True)

The example below computes the Edit distance between `'eCornell'` and a sentence starting with `'Cornell...'`. Since the first word of each string differs only by `'e'`, the leftmost submatrix has mostly low counts, with the lowest count of 1 staying approximately on the diagonal. 

After progressing past `'Cornell'`, the edit counts begin to grow since it takes an increasing number of deletions to transform the shorter string `'eCornell'` into a longer phrase.

In [ ]:
plt.figure(figsize=(20,3))
EditDist('eCornell', 'Cornell University in Ithaca, New York', ShowMatrix=True);

The following example compares two sequences using word elements instead of character elements. The bottom right corner of the DP matrix shows that only one edit is needed to make the sentences equal: adding or deleting `'University'`.

In [ ]:
EditDist('We study at Cornell'.split(), 'We study at Cornell University'.split(), ShowMatrix=True)

The next example compares the distance between two strings of binary code. It may be less intuitive to compare these sequences manually, but the algorithm does the same amount of work regardless of the representation of the elements. The bottom right corner shows that one edit operation is needed. You can also see that it's required in the third position, since that's where the matrix's diagonal element switches from 0 to 1.

In [ ]:
EditDist('0101001010', '0111001010', ShowMatrix=True)

This final example compares two DNA strings or sequences of nucleotides: A,C,G,T. This can be done to investigate the similarity between two DNA strings or to find the location of a mutation (which is an accidental change in nucleotides).

In [ ]:
EditDist('ACGTACGT', 'ACGTTTTACGT', ShowMatrix=True)

<hr style="border-top: 2px solid #606366; background: transparent;">

# **Optional Practice**
 
Now, equipped with concepts and tools, let's try to tackle a few related tasks.


As you work through these tasks, check your answers by running your code in the *#check solution here* cell, to see if you’ve gotten the correct result. If you get stuck on a task, click the See **solution** drop-down to view the answer.

## Task 1

Consider directions North, South, East, and West corresponding to letters N,S,E,W, respectively. To get from point A to point B, one can either travel in a direction `'EEENNN'` or `'ENENEN'`. Where each letter indicates a passage of 1 mile distance. Compute the Edit distance between the two direction paths. 

<b>Hint:</b> Call <code>EditDist</code> with the strings corresponding to directions.

In [ ]:
# check solution here

<hr>
<font color=#606366>
    <details><summary><font color=#B31B1B>▶ </font>See <b>solution</b>.</summary>
<pre>
EditDist('EEENNN', 'ENENEN', ShowMatrix=True)
</pre>
</details> 
</font>

## Task 2

The cell below already includes a line for downloading NLTK's `'names'` corpus. Use the file `female.txt` to identify all traditionally feminine names in this corpus. When lowercased, how many of these names are exactly two edits away from `'maria'`?

<b>Hint:</b> You need to build a loop or list comprehension, where each lower cased name is compared to the query word 'maria'. Any names that are too long or too short can be dropped, so that the final list contains only names which are exactly two edit operations away from 'maria'.

In [ ]:
_ = nltk.download(['names'], quiet=True)   # download NLTK's name corpus

# check solution here

<hr>
<font color=#606366>
    <details><summary><font color=#B31B1B>▶ </font>See <b>solution</b>.</summary>
<pre>
AllComps = sorted([(EditDist(s.lower(), 'maria'),s) for s in nltk.corpus.names.words('female.txt')], reverse=False)
CompsD2 = [s for d,s in AllComps if d==2]  # drop names that are too long or too short
print(f'There are {len(CompsD2)} matches')
'|'.join(s for d,s in AllComps if d==2)
</pre>
</details> 
</font>